# Code for the post

Code is from below links with minor changes made for the blog post.

References: 
    https://gist.github.com/karpathy/d4dee566867f8291f086
    
    https://gist.github.com/satyajitvg/9a5f782ccef5ff81f7f9863b62218b06
    
        

In [140]:
import numpy as np

def random_init(num_rows, num_cols):
    return np.random.rand(num_rows, num_cols)*0.01

def zero_init(num_rows, num_cols):
    return np.zeros((num_rows, num_cols))

class DataReader:
    def __init__(self, path, seq_length):
        self.fp = open(path, "r")
        #self.data = "some really long text to test this. maybe not perfect but should get you going."#self.fp.read()
        self.data = self.fp.read()
        #find unique chars
        chars = list(set(self.data))
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()


In [220]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        self.U = random_init(hidden_size, vocab_size) # input to hidden
        self.W = random_init(hidden_size, hidden_size) # hidden to hidden
        self.V = random_init(vocab_size, hidden_size) # hidden to output
        self.b = zero_init(hidden_size, 1) # bias for hidden layer
        self.c = zero_init(vocab_size, 1) # bias for output
        
        # memory vars for adagrad, 
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
        
    def forward(self, inputs, hprev):
            xs, hs, os, ycap = {}, {}, {}, {}
            hs[-1] = np.copy(hprev)
            for t in range(len(inputs)):
                xs[t] = zero_init(self.vocab_size,1)
                xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
                hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b) # hidden state
                os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
                ycap[t] = self.softmax(os[t]) # probs for next char
            return xs, hs, ycap
        
    def backward(self, xs, hs, ps, targets):
            # backward pass: compute gradients going backwards
            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):
                dy = np.copy(ps[t])
                #through softmax
                dy[targets[t]] -= 1 # backprop into y
                #calculate dV, dc
                dV += np.dot(dy, hs[t].T)
                dc += dc
                #dh includes gradient from two sides, next cell and current output
                dh = np.dot(self.V.T, dy) + dhnext # backprop into h
                # backprop through tanh non-linearity 
                dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
                db += dhrec
                #calculate dU and dW
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                #pass the gradient from next cell to the next iteration.
                dhnext = np.dot(self.W.T, dhrec)
            # clip to mitigate exploding gradients
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam) 
            return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
            """loss for a sequence"""
            # calculate cross-entrpy loss
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = zero_init(self.vocab_size, 1)
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = zero_init(self.vocab_size,1)
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0
            threshold = 0.01
            smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.seq_length
            while (smooth_loss > threshold):
                if data_reader.just_started():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                smooth_loss = smooth_loss*0.999 + loss*0.001
                hprev = hs[self.seq_length-1]
                if not iter_num%500:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
                iter_num += 1

    def predict(self, data_reader, start, n):

        #initialize input vector
        x = zero_init(self.vocab_size, 1)
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = zero_init(self.vocab_size,1)
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [221]:
seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("input.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

efoorlotxaymt.tlextg to lycllroolrtmd grny aetrbtrld met lytntmt. gap tdldaisontllaggoa  saxyomxgdc  grtetxtegoebtunu ypxbb.tresroxemtn nlxs ogoeut ugoirrleyram rsno .reypsegetcroeoroboabem xerol lfot


iter :0, loss:77.276060
fdbaddnmd.cphssfddrib.rpxlpumaolpudsulamlpyipubdtbdhdd.cdulafndpnsaephrsxryxia.xdhd armuxbmnsprmulp.naanaul iccplcafdads.lfamfumpmlaammmmcfxsflmadmanbxr.andurxhcs.smmlcmnp.dpurburbhrcmrbfmmdb urmpmsfl


iter :500, loss:69.476174
 tooou t lyyieeralbots.oipect noneabep bot to t lou to to eylhi pessnept pe go sest nou on t iuhohisun pn yoopxugoooxpxt tenot tobto shngo .o goouu te aobpese buu go t  go to gomapby lyeeslyou aycst .


iter :1000, loss:52.577288
o terfest yong text to test mally t gep not to test yonlut tect not goie not goit yong gext t t. maybe not seit you pect but to to t but goigint thishis. maybe not goig nshould ge ret you pect not goi


iter :1500, loss:36.617771
t yong text to t but perfectes. mso test you goixt should get nou sh goit yong tet y

hould get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goie not perfect but 


iter :18000, loss:0.074122
t you goixt should get you goie not perfect but should get you goit you goixt should get you goie not perfect but should get you goit you goie not perfect but should get you goixt should get you goit 


iter :18500, loss:0.071578
 test this. maybe not perfect but should get you goixt should get you goixt should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goie not perfect


iter :19000, loss:0.069195
ect but should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goixt  u goixt should get you goixt should get you goie not perfect but should get y


iter :19500, loss:0.066953
t you goie not perfect but should get you goixt should get you goi. not perfect 

hould get you goi text to test this. maybe not perfect but should get you goie not perfect but should get you goixt should get you should get you goist should get you goie not perfect but should get y


iter :36000, loss:0.031734
t you goie not perfect but should get you goie not perfect but should get you goit this. maybe not perfect but should get you goie not perfect but should get you goie not perfect but should get you go


iter :36500, loss:0.031240
 to test this. maybe not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goit you goie not p


iter :37000, loss:0.030760
ect but should get you goixt should get you goixt should get you goixt should get yoo goixt should get you goie not perfect but should get you goixt should get you goixt should get you goixt should ge


iter :37500, loss:0.030292
t you goixt should get you goie not perfect but should get you goie not perfect 

hould get you goie not perfect but should get you goixt should get you goixt should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goixt should ge


iter :54000, loss:0.019899
xt should get you goi text to test this. maybe not perfect but should get you goixt should get you goixt should get you goixt should get you goit this. maybe not perfect but should get you goie not pe


iter :54500, loss:0.019698
 to test this. maybe not perfect but should get you goie not perfect but sybe not perfect but should get you goie not perfect but should get you goixt should get you goie not perfect but should get yo


iter :55000, loss:0.019502
hould get you goi text to test this. maybe not perfect but should get you goie not perfect but should get you goie not perfect but should get you goixt should get you goie not perfect but should get y


iter :55500, loss:0.019310
t this. maybe not perfect but should get you goie not perfect but should get you

tsxt should get you goixt should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfec


iter :72000, loss:0.014403
e not perfect but should get you goie not perfect but should get you goie not perfect but u g text to test this. maybe not perfect but should get you goie not perfect but should get you goixt should g


iter :72500, loss:0.014291
 to test this. maybe not perfect but should get you goixt should get you goixt should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get yo


iter :73000, loss:0.014180
hbut should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you goie not perfect but should get you 


iter :73500, loss:0.014069
t this. maybe not perfect but should get you goie not perfect but should get you

ect but should get you goit this. maybe not perfect but should get you goie not perfect but should get you goially long text to test this. maybe not perfect but should get you goially long text to tes


iter :90000, loss:0.011091
xt should get you goie not perfect bto test this. maybe not perfect but should get you goixt perfect but should get you goit this. maybe not perfect but should get you goie not perfect but should get 


iter :90500, loss:0.011015
 to test this. maybe not perfect but should get you goit you goit this. maybe not perfect but should get you goie not perfect but should get you goit this. maybe not perfect but should get you goit th


iter :91000, loss:0.010940
ect but should get you goixt should get you goit you goit this. maybe not perfect but should get you goie not perfect but should get you goi text to test this. maybe not perfect but should get you goi


iter :91500, loss:0.010865
t this. maybe not perfect but should get you goixt should get you goit you goie 

In [225]:
rnn.predict(data_reader, 'get', 50)

'get text to test this. maybe not perfect but should g'